# `main.cpp` Yapısı ve Akış Açıklaması

Bu not defteri, **PerceptionProject** içindeki `main.cpp` dosyasının yapısını ve diğer dosyalar (`perception_types.hpp`, `perception_node.hpp`, `perception_node.cpp`) ile olan ilişkisini adım adım açıklamak için hazırlanmıştır.

Amaç:

- `main.cpp` dosyasının **uygulama seviyesi akışını** anlamak,
- `ImageFrame`, `PerceptionConfig` ve `PerceptionNode` tiplerinin gerçek kullanımını görmek,
- `perception.process(frame)` çağrısının dışarıdan nasıl göründüğünü kavramak.

## 1. Dosya Başlangıcı – `#include` Satırları

```cpp
#include <iostream>
#include <opencv2/opencv.hpp>

#include "perception_types.hpp"
#include "perception_node.hpp"
```

Bu bölümde:

- `#include <iostream>`  
  - Konsola çıktı vermek (`std::cout`, `std::cerr`) için kullanılır.
- `#include <opencv2/opencv.hpp>`  
  - OpenCV'nin ana header'ı.  
  - `cv::Mat`, `cv::imread`, `cv::imshow` gibi fonksiyonlar/typename'ler için gereklidir.
- `#include "perception_types.hpp"`  
  - `ImageFrame`, `DetectedObject`, `PerceptionConfig` struct'larını kullanmak için.
- `#include "perception_node.hpp"`  
  - `PerceptionNode` sınıfını kullanmak için.

Bu `#include` satırları sayesinde, main.cpp dosyası:
- Hem OpenCV fonksiyonlarına,
- Hem de kendi algılama modülünün veri tipleri ve node sınıfına erişebilir hale gelir.

## 2. `main` Fonksiyonunun İmzası

```cpp
int main()
{
    // ...
}
```

- Programın giriş noktasıdır.
- Uygulama çalıştırıldığında ilk olarak `main()` fonksiyonu çağrılır.
- `int` dönüş tipi:
  - `0` → başarıyla tamamlandı,
  - `0` dışı → hata kodu (örneğin `1`), hatayı işletim sistemine bildirir.

## 3. Adım 1 – Test Görüntüsünü Yükleme

```cpp
cv::Mat img = cv::imread("test.jpg");
if (img.empty())
{
    std::cerr << "[main] test.jpg yüklenemedi. "
              << "Çalışma dizinini ve dosya yolunu kontrol et.
";
    return 1;
}
```

### Açıklama

- `cv::Mat img = cv::imread("test.jpg");`
  - Diskten **test.jpg** isimli dosyayı okumaya çalışır.
  - Sonuç, OpenCV'nin temel görüntü veri tipi olan `cv::Mat` içine alınır.
- `if (img.empty())`:
  - Dosya bulunamazsa veya okunamazsa `img` boş (`empty`) olur.
  - Böyle bir durumda:
    - Hata mesajı `std::cerr` ile ekrana yazılır,
    - `return 1;` ile program **hata kodu** ile sonlandırılır.

### Neden bu adımı yapıyoruz?

Bu adım, Uygulama 1'de **algılama iskeletini** test etmek için basit bir giriş sağlar:

- Kendi kameramıza bağlanmak zorunda değiliz,
- Sabit bir resim üzerinden pipeline'ı test ederiz,
- `ImageFrame` ve `PerceptionNode` yapısının doğru çalışıp çalışmadığını kolayca gözlemleriz.

## 4. Adım 2 – `ImageFrame` Oluşturma

```cpp
ImageFrame frame(
    img,              // görüntü
    1,                // frame_id
    "front_camera"    // kamera adı
);
```

### Açıklama

Bu satır ile `ImageFrame` struct'ının **parametreli constructor'ı** kullanılır:

- `img` → `cv::Mat` görüntü, frame'in ham piksel verisi.
- `1` → `frame_id`, bu frame için verilen ID (veya sayaç).
- `"front_camera"` → `camera_name`, bu görüntünün hangi kameradan geldiğini belirtir.

Struct içinde ayrıca:

- `timestamp` alanı `std::chrono::steady_clock::now()` ile otomatik olarak **oluşturulma zamanını** kaydeder.

### Neden böyle yapıyoruz?

- `ImageFrame`, tek bir kamera karesini **veri + bağlam** halinde paketler:
  - Görüntü (`image`),
  - Frame kimliği (`frame_id`),
  - Zaman damgası (`timestamp`),
  - Kamera adı (`camera_name`).

Bu sayede `PerceptionNode::process()` fonksiyonuna **sadece bir parametre** vererek tüm bu bilgileri taşıyabiliyoruz:
```cpp
auto detections = perception.process(frame);
```

## 5. Adım 3 – `PerceptionConfig` Doldurma

```cpp
PerceptionConfig config;

// Şimdilik model_path boş kalsın: Uygulama 1'de dummy inference kullanıyoruz.
config.model_path = "";      // İleride: "yolov8s.torchscript.pt" vb. vereceğiz.

// Cihaz seçimi: "cpu" veya "cuda"
config.device = "cpu";

// Modelin beklediği giriş boyutu (Uygulama 1'de sadece log için kullanıyoruz)
config.input_width  = 640;
config.input_height = 640;

// Normalize parametreleri (şimdilik nötr bırakıyoruz)
config.mean = {0.0f, 0.0f, 0.0f};
config.std  = {1.0f, 1.0f, 1.0f};

// Sınıf isimlerini şimdiden doldurmak istersen:
// Dummy bir örnek set verebiliriz.
config.class_names = {
    "dummy_object"   // class_id = 0 için
};
```

### Açıklama

Bu blok, algılama nodunun çalışma şeklini belirleyen **ayar paketini** doldurur:

- `config.model_path`:
  - Uygulama 1'de boş, çünkü gerçek model yüklemek zorunda değiliz.
  - İleride TorchScript dosyasının yolu buraya yazılacak.
- `config.device`:
  - `"cpu"` veya `"cuda"` olabilir.
  - Constructor içinde bu bilgi kullanılarak `torch::Device` seçilir.
- `config.input_width` / `config.input_height`:
  - Modelin beklediği giriş boyutudur.
  - Uygulama 1'de yalnızca log amaçlıdır.
- `config.mean` ve `config.std`:
  - Normalizasyon için kullanılacak parametrelerdir.
  - Şimdilik nötr bırakılmıştır.
- `config.class_names`:
  - `class_id` → `class_name` dönüşümü için kullanılır.
  - Dummy modda sadece `"dummy_object"` eklenmiştir.

### Neden bu adımı yapıyoruz?

`PerceptionConfig`, `PerceptionNode`'un davranışını belirleyen **konfigürasyon paketidir**.  
Node'u her seferinde kodu değiştirmeden, sadece config ile yönetebilmek için bu yapı kullanılır.

## 6. Adım 4 – `PerceptionNode` Oluşturma

```cpp
PerceptionNode perception(config);
```

### Açıklama

Burada:

- `PerceptionNode` sınıfından `perception` isimli bir nesne oluşturulur.
- Constructor'a `config` verilir.

Constructor içinde tipik olarak:

1. `config_` üyesi doldurulur,
2. `device_` (CPU / CUDA) seçilir,
3. `config_.model_path` boş değilse TorchScript modeli yüklemeye çalışılır,
4. `model_loaded_` bayrağı güncellenir.

### Neden bu adımı yapıyoruz?

Bu adımla birlikte:

- Algılama modülü artık **hazır bir node nesnesi** olarak ayağa kalkar.
- Dış dünyadan sadece `perception.process(frame)` çağrısıyla kontrol edilebilir hale gelir.

## 7. Adım 5 – `process(frame)` Çağrısı

```cpp
auto detections = perception.process(frame);
```

### Açıklama

Bu satır ile:

- `frame` → `ImageFrame` (görüntü + bağlam),
- `perception` → `PerceptionNode`,
- `detections` → `std::vector<DetectedObject>` elde edilir.

İçerideki akış:

1. `preprocess(frame)` çağrılır,
2. `inference(input_tensor)` çağrılır,
3. `postprocess(frame, output_tensor)` çağrılır,
4. Sonuçta `DetectedObject` listesi döner.

**Uygulama 1'de:**

- Bu fonksiyonun içi dummy implementasyon ile çalışır:
  - `preprocess()` → sadece log basar ve boş tensor döner,
  - `inference()` → model yoksa dummy tensor döner,
  - `postprocess()` → sahte bir `DetectedObject` oluşturur.

Amaç: İskeletin doğru çalıştığını ve akış sırasını görmek.

## 8. Adım 6 – Sonuçların Yazdırılması

```cpp
std::cout << "\n[main] Dönüşte gelen tespitler:\n";

if (detections.empty())
{
    std::cout << "  (Tespit yok)\n";
}
else
{
    for (const auto& obj : detections)
    {
        std::cout << "  class_id: " << obj.class_id
                  << ", name: "   << obj.class_name
                  << ", score: "  << obj.score
                  << ", bbox: ("  << obj.x << ", " << obj.y
                  << ", "         << obj.width << ", " << obj.height
                  << "), track_id: " << obj.track_id
                  << "\n";
    }
}
```

### Açıklama

- Önce bir başlık satırı yazılır: `"[main] Dönüşte gelen tespitler:"`.
- Eğer `detections` listesi boşsa:
  - `(Tespit yok)` mesajı yazılır.
- Aksi halde:
  - Her `DetectedObject` için:
    - `class_id`
    - `class_name`
    - `score`
    - `bbox` (x, y, width, height)
    - `track_id`  
    gibi alanlar ekrana yazılır.

### Neden bu adımı yapıyoruz?

Bu adım sayesinde:

- `PerceptionNode`'un dışarıya ne tür bir çıktı verdiğini **somut olarak** görürüz.
- `DetectedObject` yapısının gerçek dünyada (loglarda) nasıl görüneceğini kavrarız.
- Uygulama 1'de dahi, akışın sonunda dışarıya düzgün bir tespit listesi çıktığını teyit ederiz.

## 9. `return 0;` veya `return 1;`

```cpp
return 0;
```

- `main` fonksiyonunun sonunda tipik dönüş değeri:
  - `0`: Programın başarıyla tamamlandığını ifade eder.
  - `1`: Bir hata meydana geldiğini (örneğin resim yüklenemedi) ifade eder.

Bu, işletim sistemine programın durumu hakkında bilgi verir ve özellikle script/otomasyon ortamlarında önemlidir.

## 10. Genel Özet

`main.cpp`, PerceptionProject'in **uygulama tarafını** temsil eder:

1. Test resmi yüklenir (`cv::imread`).
2. Bu resimden bir `ImageFrame` oluşturulur.
3. `PerceptionConfig` doldurularak node'un çalışma ayarları belirlenir.
4. `PerceptionNode` nesnesi (`perception`) bu config ile ayağa kaldırılır.
5. `perception.process(frame)` çağrısı ile algılama pipeline'ı tetiklenir.
6. Dönen `std::vector<DetectedObject>` sonuçları ekrana yazdırılır.

Bu yapı sayesinde:

- Algılama modülü (`PerceptionNode`) ile uygulama seviyesi kod (`main.cpp`) **temiz bir şekilde ayrılmış** olur.
- `main.cpp`, sadece:
  - Girdi hazırlama (`ImageFrame`),
  - Node konfigürasyonu (`PerceptionConfig`),
  - Process çağrısı,
  - Sonuçları işleme/loglama  
  gibi görevleri üstlenir.

Algılama mantığının detayları ise tamamen `PerceptionNode` içinde kalır.
